<a href="https://colab.research.google.com/github/arnaldourbina/HACKATON_Equipo_H12-25-L-Equipo74/blob/main/notebook_flight_on_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Célula 1: imports y carga de datos
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import joblib


In [11]:
# Carga del dataset
# Debe tener columnas: aerolinea, origen, destino, fecha_partida, distancia_km, retrasado
df = pd.read_csv('flights2015.csv')  # colocar el archivo en ds/data/

/tmp/ipython-input-1991655639.py:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('flights2015.csv')  # colocar el archivo en ds/data/


In [12]:
#Célula 2: limpieza y features
# Convertimos fecha_partida a datetime y extraemos hora y día de semana
df['fecha_partida'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])
df['hora'] = df['fecha_partida'].dt.hour
df['dia_semana'] = df['fecha_partida'].dt.dayofweek  # 0=lunes, 6=domingo

In [15]:
# Definimos X e y
# Crear la columna 'retrasado' basada en 'ARRIVAL_DELAY'
df['retrasado'] = (df['ARRIVAL_DELAY'] > 0).astype(int)

X = df[['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'hora', 'dia_semana', 'DISTANCE']]
y = df['retrasado']  # 0 puntual, 1 retrasado

In [25]:
# Célula 3: pipeline y entrenamiento
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import joblib

def to_string_transformer(x):
    return x.astype(str)

cat_features = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
num_features = ['hora', 'dia_semana', 'DISTANCE']

preprocess = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('to_string', FunctionTransformer(to_string_transformer, validate=False)),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), cat_features),
        ('num', 'passthrough', num_features),
    ]
)

model = LogisticRegression(max_iter=1000)

pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('clf', model)
])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

pipeline.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('to_string',
                                                                   FunctionTransformer(func=<function to_string_transformer at 0x7ff0fd6a49a0>)),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['AIRLINE', 'ORIGIN_AIRPORT',
                                                   'DESTINATION_AIRPORT']),
                                                 ('num', 'passthrough',
                                                  ['hora', 'dia_semana',
                                                   'DISTANCE'])])),
                ('clf', LogisticRegression(max_iter=1000))])

In [22]:
# Célula 4: evaluación
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.6417706922743801
Precision: 0.5135229251305862
Recall: 0.02119895826095707
F1: 0.04071705832138773
              precision    recall  f1-score   support

           0       0.64      0.99      0.78    746437
           1       0.51      0.02      0.04    417379

    accuracy                           0.64   1163816
   macro avg       0.58      0.50      0.41   1163816
weighted avg       0.60      0.64      0.51   1163816



In [27]:
import joblib

In [26]:
# Célula 5: exportación del modelo
import os
if not os.path.exists('model'):
    os.makedirs('model')
joblib.dump(pipeline, 'model/flight_delay_model.joblib')
print("Modelo guardado en model/flight_delay_model.joblib")

Modelo guardado en model/flight_delay_model.joblib


In [29]:
# Célula 6: helper de inferencia local (opcional)
def predecir_registro(aerolinea, origen, destino, fecha_partida, distancia_km):
    fecha = pd.to_datetime(fecha_partida)
    hora = fecha.hour
    dia_semana = fecha.dayofweek
    X_one = pd.DataFrame([{
        'AIRLINE': aerolinea, 'ORIGIN_AIRPORT': origen, 'DESTINATION_AIRPORT': destino,
        'hora': hora, 'dia_semana': dia_semana, 'DISTANCE': distancia_km
    }])
    proba = pipeline.predict_proba(X_one)[0, 1]
    pred = int(proba >= 0.5)
    return {'prevision': 'Retrasado' if pred == 1 else 'Puntual', 'probabilidad': float(proba)}

print(predecir_registro("AZ", "GIG", "GRU", "2025-11-10T14:30:00", 350))

{'prevision': 'Puntual', 'probabilidad': 0.3411009586662316}
